# Inteligencia Artificial 2
## P1 Project: Implement a Recommender system using a bipartite network projection
### Autores: Miguel Brito, Diana Cuenca, José Escudero, Danny Huacon, Steveen Terán

En el presente informe, se abordará la implementación de un sistema de recomendación con el uso de una red bipartita. Esta es una técnica que permite analizar relaciones complejas entre dos conjuntos de elementos. Por ello, se analizará y proyectara una red de productos en funcion de las recomendaciones emitidas por los usuarios. Lo que permitirá identificar patrones y conexiones significativas entre los productos, lo que facilitará la generación de recomendaciones personalizadas.

## Importar y analizar datos
#### 1. Importar librerías

In [ ]:
# Importaciones básicas

import numpy as np
import pandas as pd


Se hace uso de **ast_node_interactivity** con el fin de inspeccionar todos los resultados que llegue a generar una celda, y no solo el último resultado. Por ello, se configura el notebook con la expresión _"all"_

In [ ]:
# Configuración específica de Jupyter Notebook

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Librerias de visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns


#### 2. Importar datos

Se lee el archivo de los datos y se carga en un dataframe, asi también se especifica los nombres de las columnas, y se muestran las priumeras 5 filas. 

_Cabe aclarar que se eliminó la columna timestamp con el fin de reducir la dimensionalidad de los datos, además que no se vió la relevancia para el objetivo y posterior análisis de este informe._

In [ ]:
# Cargar datos desde un archivo CSV y asignar nombres a las columnas
electronics_data = pd.read_csv("./ratings_electronics.csv", names=['userId', 'productId', 'Rating', 'timestamp'])

# Eliminar la columna 'timestamp' de los datos
electronics_data.drop(['timestamp'], axis=1,inplace=True)

# Mostrar las primeras filas del DataFrame
electronics_data.head()


#### 3. Analizar rango de ratings y revisar integridad de datos

Se realiza una comprensión inicial de la distribución de las calificaciones de los usuarios hacia los productos, con el fin de poder identificar posibles problemas en la integridad de los datos y corregir estos errores antes de la contrucción del modelo.

In [ ]:
# Encontrar las calificaciones mínima y máxima

print('Minimum rating is: %d' %(electronics_data.Rating.min()))
print('Maximum rating is: %d' %(electronics_data.Rating.max()))
print("=====================================================================================================")
print('Number of missing values across columns: \n',electronics_data.isnull().sum())


#### 4. Graficar distribución de ratings

Se crea un diccionario que cuenta la frecuencia de cada valor único del **Rating**. Esto se muestra en un gráfico de barras para visualizar la naturaleza de los datos, en este caso, la distribución de las calificaciones de los usuarios.

In [ ]:
# Create a dictionary in which the key is the rating, and the value is the number of times that rating occurs
rating_distribution = electronics_data.groupby(['Rating'])['Rating'].agg(['count'])

# Check the distribution of the rating
sns.barplot(x=rating_distribution.index, y=rating_distribution['count'])


#### 5. Imprimir cantidad de ratings, usuarios unicos y productos unicos

Se imprime la cantidad total de calificaciones, la cantidad de usuarios únicos y la cantidad de productos únicso en el conjunto de datos **electronics_data**. Con ello, podemos obtener información clave sobre la magnitud y diversidad de los usuarios y los productos recomendados.

In [ ]:
print("Total no of ratings :",electronics_data.shape[0])
print("Total No of Users   :", len(np.unique(electronics_data.userId)))
print("Total No of products  :", len(np.unique(electronics_data.productId)))


## Desarrollo del Modelo

#### 1. Generar Red Bipartita

Para el presente sistema de recomebntación, es necesaria la generación de una red bipartita que capture las relaciones entre los usuarios y los productos.

En versiones anteriores de NetworkX, la creación de esta red se realizaba utilizando el método **`from_pandas_dataframe`**. Sin embargo, con el avance de las versiones y la evolución de la biblioteca, este método ha sido marcado como obsoleto, y se recomienda el uso de **`from_pandas_edgelist`** como la alternativa actualizada y preferida.

_Fuente: `https://stackoverflow.com/a/49580740`_

Se utiliza la librería **networkx**, para la creación, manipulación y estudio de estructuras, funciones y dinámicas de redes complejas. De esta se utilizan las siguientes herrameintas para el modelo de recomentacion y la construcción de la red bipartita:

- **bipartite** : utilizada para trabajar una red bipartita, con lo que podemos manipular los grafos asi como tener funciones y algoritmos para realizar operaciones o proyecciones en dicha red.

- **itemgetter** : utilizada para extraer elementos específicos de una secuencia. En este caso, para seleccionar nodos según criterios configurables.

In [ ]:
# Modelado de la red bipartita

import networkx as nx
from networkx.algorithms import bipartite
from operator import itemgetter


A continuación, se crea un nuevo dataframe con el fin de almacenar los productos que contienen calificación de 5 estrellas unicamente, luego se crea i grafo, donde los nodos son los usuarios y los productos, asi como los bordes representan las interacciones entre usuarios y productos con calificación de 5 estrellas.

Para finalizar, se aginan etiquetas a cada dato recopilado en el grafo y se comprueba si es un grafo bipartito con **`is_bipartite`**.

In [ ]:
# Filtrar las revisiones con 5 estrellas
five_star_reviews = electronics_data[electronics_data['Rating'] == 5]

# Crear el grafo bipartito
B = nx.from_pandas_edgelist(five_star_reviews, 'userId', 'productId', edge_attr=None, create_using=None)

# Assign label to nodes
unique_user_ids = set(electronics_data['userId'])
for node in B.nodes():
    if node in unique_user_ids:
        B.nodes[node]['bipartite'] = 'users'
    else:
        B.nodes[node]['bipartite'] = 'products'

# Confirmar que es bipartito
bipartite.is_bipartite(B)


#### 2. Ejemplo de sistema de recomendacion con datos de la red bipartita. 

    a. En este bloque, se presenta un ejemplo concreto de cómo implementar un sistema de recomendación basado en una red bipartita. El enfoque adoptado consiste en seleccionar un usuario específico y obtener recomendaciones de productos basándose en la actividad de otros usuarios que han revisado productos similares.

Se inicia seleccionando un usuario del conjunto de datos, para luego identificar los revisados por dicho usuario. Así, se buscan los usuarios que han revisado los mismos productos que el usuario en cuestión. Esta información forma la base para generar recomendaciones de productos para el usuario seleccionado.

In [ ]:
# Combinación Listas
from itertools import chain

# Obtener un usuario del dataframe
user_node = five_star_reviews.iloc[0]['userId']

# Obtener los productos que ha revisado este usuario
products_reviewed_by_user = B.neighbors(user_node)

# Obtener los usuarios que han hecho review de los mismos productos que el usuario en cuestión
related_users = list(bipartite.weighted_projected_graph(B, [user_node]))

# Obtener los productos que han sido revisados por los usuarios relacionados
recommendations = chain.from_iterable([list(B.neighbors(node)) for node in related_users])

# Eliminar los productos que ya ha revisado el usuario en cuestión
recommendations = set(recommendations) - set(products_reviewed_by_user)

recommendations



    b. En este bloque se ordena los productos obtenidos en el bloque anterior, de acuerdo a la cantidad de usuarios que han revisado el producto, y se selecciona los 10 productos con mayor cantidad de usuarios que lo han revisado

Se crea una lista de tuplas donde cada una contiene un producto y el número de usuarios que lo han revisado. Esta lista se ordena en base al conteno de recomentdaciones de manera descendente, y se seleccionan las 10 principales.Por último, se itera las mejores recomendaciones y se imprime el producto y la cantidad de usuarios que se interesaron por el producto.

In [ ]:
# Lista de tuplas con productos y su conteo de recomendaciones
product_recommendation_counts = [(product, B.degree(product)) for product in recommendations]

# Ordenar la lista por conteo de recomendaciones de manera descendente
top_recommended_products = sorted(product_recommendation_counts, key=itemgetter(1), reverse=True)[:10]

# Imprimir las principales recomendaciones
for product, count in top_recommended_products:
    print(f"Producto: {product}, revisado por: {count} usuarios")


#### 3. Construcción de un Grafo Bipartito con un Subconjunto de Datos

    a. En esta etapa del análisis, se procede a la creación de un grafo bipartito, tomando como punto de partida un subconjunto específico de datos compuesto por las primeras 100 revisiones con calificación de 5 estrellas.


Se selecciona las primeras 100 revisiones con calificación máxima, teniendo asi un conjunto representativo para la construcción del grafo. A partir de este, se construye un grafo bipartito donde los nodos representan usuarios y productos, y los bordes reflejan las interacciones entre ellos. Se asignan etiquetas para facilitar la interpretación del grafo. Y, por último, se verifica que el grafo tenga una estructura bipartita.

In [ ]:
# Visualizar una parte del gráfico
subset_data = five_star_reviews.head(100)

# Crear un grafo bipartito basado en un subconjunto de datos
B_sub = nx.from_pandas_edgelist(subset_data, 'userId', 'productId', edge_attr=None, create_using=None)

# Asignar etiquetas a los nodos
unique_user_ids = set(electronics_data['userId'])
for node in B_sub.nodes():
    if node in unique_user_ids:
        B_sub.nodes[node]['bipartite'] = 'users'
    else:
        B_sub.nodes[node]['bipartite'] = 'products'

# Confirmar que es un grafo bipartito
bipartite.is_bipartite(B)


    b. En este bloque, se imprime y visualiza el grafo bipartito generado, con el fin de proporcionar una representación gráfica que permita entender la estructura de la red y las interacciones entre usuarios y productos.

Se utiliza **bipartite_layout** para calcular una disposición específica del grafo, se asignan colores para diferenciar usuarios y productos asi como leyendas. Por último, cabe destacar que este gráfico nos ayuda a tener una idea visual sobre las interacciones entre usuarios y productos.

In [ ]:

# Calcular la disposición específica para nodos
pos = nx.bipartite_layout(B_sub, nodes=[n for n, d in B_sub.nodes(data=True) if d['bipartite'] == 'users'])

# Asignar colores a los nodos según su bipartición (azul para usuarios, rojo para productos)
colors = ["blue" if d['bipartite'] == 'users' else "red" for n, d in B_sub.nodes(data=True)]

# Configurar el tamaño y la presentación del gráfico
plt.figure(figsize=(12, 12))

# Dibujar el grafo bipartito
nx.draw(B_sub, pos=pos, node_color=colors, with_labels=False, node_size=20)

# Crear leyenda - nodos de usuarios y productos
blue_patch = plt.Line2D([0], [0], marker='o', color='w', label='Usuarios', markersize=10, markerfacecolor='blue')
red_patch = plt.Line2D([0], [0], marker='o', color='w', label='Productos', markersize=10, markerfacecolor='red')
plt.legend(handles=[blue_patch, red_patch])
plt.title("Subconjunto de la Red Bipartita Usuario-Producto")
plt.show()


#### 4. Poda de grafo bipartito e impresión del mismo

    a. En esta etapa, se realiza una poda selectiva del grafo bipartito con el fin de obtener los usuarios que hayan hecho más de 250 reviews.

Primero, se empieza podando todos aquellos usuarios que posean un umbral inferior a 250 y se contruye un subgrafo a partir de dicha poda. A continuación y a partir del subgrafo, excluimos todos aquellos productos que no posean niguna recomendación y nuevamente se construye un subgrafo. Contamos el número de usuario y productos del grafo con poda selectiva.

In [ ]:
# Definir umbral de grado para usuarios
degree_threshold = 250

# Identificar usuarios con grado inferior al umbral
users_nodes_to_remove = [node for node, data in B.nodes(data=True) if data['bipartite'] == 'users' and B.degree(node) < degree_threshold]

# Crear un subgrafo excluyendo los usuarios identificados
trimmed_B = B.subgraph(set(B.nodes) - set(users_nodes_to_remove))

# Identificar productos sin conexiones con usuarios
orphan_products_nodes = [node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'products' and trimmed_B.degree(node) == 0]

# Crear un subgrafo excluyendo los productos huérfanos
trimmed_B = trimmed_B.subgraph(set(trimmed_B.nodes) - set(orphan_products_nodes))

# Contar el número de usuarios y productos
users_count = len([node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'users'])
products_count = len([node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'products'])

print(f"El grafo podado tiene {users_count} usuarios y {products_count} productos")


Visualización del grafo bipartito podado

In [ ]:
# Dibujar el grafo bipartito podado
pos = nx.bipartite_layout(trimmed_B, nodes=[n for n, d in trimmed_B.nodes(data=True) if d['bipartite'] == 'users'])
colors = ["blue" if d['bipartite'] == 'users' else "red" for n, d in trimmed_B.nodes(data=True)]

plt.figure(figsize=(12, 12))
nx.draw(trimmed_B, pos=pos, node_color=colors, with_labels=False, node_size=20)

# Crear leyenda para distinguir nodos de usuarios y productos
blue_patch = plt.Line2D([0], [0], marker='o', color='w', label='Usuarios', markersize=10, markerfacecolor='blue')
red_patch = plt.Line2D([0], [0], marker='o', color='w', label='Productos', markersize=10, markerfacecolor='red')
plt.legend(handles=[blue_patch, red_patch])
plt.title("Subconjunto del Grafo Bipartito Usuarios-Productos después de la Poda")
plt.show()


    b. En esta etapa, se realiza una poda selectiva del grafo bipartito con el fin de obtener los productos que poseen más de 10000 reviews.

Primero, se empieza popdando todos los productos que posean un umbral inferior a 10000 y se contruye un subgrafo a partir de dicha poda. A continuación y a partir del subgrafo, excluimos todos aquellos usuarios que no tienen conexiones y se construye un nuevo subgrafo. Contamos el número de usuario y productos del grafo con poda selectiva.

In [ ]:
# Definición del umbral de grado para conservar nodos de productos
degree_threshold = 10000

# Identificación y eliminación de nodos de productos con grado inferior al umbral
product_nodes_to_remove = [node for node, data in B.nodes(data=True) if data['bipartite'] == 'products' and B.degree(node) < degree_threshold]
trimmed_B = B.subgraph(B.nodes - set(product_nodes_to_remove))

# Identificación y eliminación de nodos de usuarios huérfanos en el grafo podado
orphan_user_nodes = [node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'users' and trimmed_B.degree(node) == 0]
trimmed_B = trimmed_B.subgraph(trimmed_B.nodes - set(orphan_user_nodes))

# Conteo de usuarios y productos en el grafo podado
users_count = len([node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'users'])
products_count = len([node for node, data in trimmed_B.nodes(data=True) if data['bipartite'] == 'products'])

print(f"El grafo podado tiene {users_count} usuarios y {products_count} productos")


Visualización del grafo bipartito podado

In [ ]:
# Dibujar el grafo
pos = nx.bipartite_layout(trimmed_B, nodes=[n for n, d in trimmed_B.nodes(data=True) if d['bipartite'] == 'users'])
colors = ["blue" if d['bipartite'] == 'users' else "red" for n, d in trimmed_B.nodes(data=True)]

plt.figure(figsize=(12, 12))
nx.draw(trimmed_B, pos=pos, node_color=colors, with_labels=False, node_size=20)
blue_patch = plt.Line2D([0], [0], marker='o', color='w', label='Usuarios', markersize=10, markerfacecolor='blue')
red_patch = plt.Line2D([0], [0], marker='o', color='w', label='Productos', markersize=10, markerfacecolor='red')
plt.legend(handles=[blue_patch, red_patch])
plt.title("Subconjunto del Grafo Bipartito Usuarios-Productos después de la poda")
plt.show()


## Funcion de recomendaciones e interfaz

Se crea una función que permite obtener un red de recomendaciones de productos a partir de un producto específico y un límite, identifica productos recomendados y construye un grafo ponderado en el que el producto original actúa como el nodo central, conectado a otros productos recomendados.

Esta función proporciona una herramienta para explorar y comprender las conexiones de recomendación en una red bipartita, permitiendo a los usuarios identificar patrones y tomar decisiones informadas sobre productos relacionados. Con ella, podremos automatizar el procceso de recomendación.

In [ ]:
# Definir una función para obtener una red de recomendaciones de productos
def get_product_recommendation_network(product, limit):
  # Obtener productos recomendados
  recommended_products = set(bipartite.weighted_projected_graph(B, [product])) - set([product])

  # Crear una lista de tuplas con productos recomendados y sus pesos
  recommended_products_with_weight = [(recommended_product, B.degree(recommended_product)) for recommended_product in recommended_products]

  # Seleccionar las mejores recomendaciones según el peso y limitar el número de recomendaciones
  top_recommendations = sorted(recommended_products_with_weight, key=lambda x: -x[1])[:limit]

  # Crear un grafo de networkX donde el producto original es el nodo central y los productos recomendados son los nodos vecinos
  # Utilizar bordes ponderados para indicar cuántos usuarios han comprado ambos productos.
  G = nx.Graph()
  G.add_node(product)  # Agregar el nodo del producto original
  for recommended_product, weight in top_recommendations:
    G.add_node(recommended_product)  # Agregar nodos de productos recomendados
    G.add_edge(product, recommended_product, weight=weight)  # Agregar bordes ponderados

  return G


Se hace uso de widgets de IPython para mostrar un campo dodne se puede seleccionar un producto específico, establecer un límite en el número de recomendaciones, y luego al hacer clic en el botón "Consultar", se muestra la representación visual de la red de recomendaciones.

De este modo, se puede tener una manera intruitica de explirar las conexiones de recomendacion de productos.

In [ ]:
import ipywidgets as widgets

def query(btn):
    product_id = product_id_widget.value
    n_limit = n_limit_widget.value
    recommendation_network = get_product_recommendation_network(product_id, n_limit)
    # Dibujar la red y mostrar los pesos de los bordes
    plt.figure(figsize=(5, 5))
    pos = nx.spring_layout(recommendation_network)
    nx.draw(recommendation_network, pos=pos, with_labels=True)
    edge_labels = nx.get_edge_attributes(recommendation_network, 'weight')
    nx.draw_networkx_edge_labels(recommendation_network, pos=pos, edge_labels=edge_labels)
    plt.show()


# Obtener nodos de B que sean partitos == 1 (productos)
product_nodes = [n for n, d in B.nodes(data=True) if d['bipartite'] == 'products'][:1000]

product_id_widget = widgets.Dropdown(
    options=product_nodes,
    value=product_nodes[0],
    description='Product ID:',
    disabled=False,
)
n_limit_widget = widgets.Dropdown(
    options=list(range(1,11)),
    value=5,
    description='N recommendations:',
    disabled=False,
)

button = widgets.Button(
    description='Consultar',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)

button.on_click(query)

product_id_widget
n_limit_widget
button


### Conlusiones

- A través del uso de una red bipartita se puede demostrar que resulta se ejecuta para modelar las interacciones, en el caso del presente informe, entre usuarios y productos. Lo cual permite aumentar la calidad del sistema de recomendación a través del uso de técnicas como la proyección ponderada y considerar umbrales o grados de interación entre los productos y usuarios.

- Así también, es importante destacar la optimizacón del modelo a través de una poda selectiva que demostró ser una estrategia correcta para mejorar la eficienica computacional sin compromenter la integridad de los datos.